# DeepBlue Threat Hunting

DeepBlue is an open source threat hunting tool for Windows EVTX logs, it can analyze either local Windows logs files (requires admin privileges) or .evtx files specified in command line arguments. The tool can be used as a command line interface with PowerShell.

Create files **whitelist.txt** and **regexes.txt**. These files can be empty, use examples in [the DeepBlue repository](https://github.com/sans-blue-team/DeepBlueCLI) for usage.

Run DeepBlue on EVTX file: `DeepBlue.ps1 logs.evtx`

There is an open dataset of EVTX logs when a specific attack was executed available in [EVTX-ATTACK-SAMPLES GitHub repository](https://github.com/sbousseaden/EVTX-ATTACK-SAMPLES). Some examples of what warnings DeepBlue raises from different MITRE ATT&CK categories below:

**Credential Access**

Phishing attempt, where a window asking the user to input their credentials is created

In [1]:
Get-ChildItem "Credential_Access\" -Filter *.evtx | 
Foreach-Object {
   .\DeepBlue.ps1 $_.FullName
}

The below script needs to be able to find the current output cell; this is an easy method to get it.


Date    : 09/09/2019 16.35.09
Log     : Powershell
EventID : 4104
Message : Suspicious Command Line
Results : Long Command Line: greater than 1000 bytes
          
Command : function Invoke-LoginPrompt{
          $cred = $Host.ui.PromptForCredential("Windows Security", "Please enter user 
          credentials", "$env:userdomain\$env:username","")
          $username = "$env:username"
          $domain = "$env:userdomain"
          $full = "$domain" + "\" + "$username"
          $password = $cred.GetNetworkCredential().password
          Add-Type -assemblyname System.DirectoryServices.AccountManagement
          $DS = New-Object System.DirectoryServices.AccountManagement.PrincipalContext([System.Dire
          ctoryServices.AccountManagement.ContextType]::Machine)
          while($DS.ValidateCredentials("$full","$password") -ne $True){
              $cred = $Host.ui.PromptForCredential("Windows Security", "Invalid Credentials, 
          Please try again", "$env:userdomain\$env:userna

**Command and Control**

RDP Tunneling via SSH

In [2]:
Get-ChildItem "Command_and_Control\" -Filter *.evtx | 
Foreach-Object {
   .\DeepBlue.ps1 $_.FullName
}


Date    : 13/02/2019 20.01.41
Log     : Security
EventID : 1102
Message : Audit Log Clear
Results : The Audit log was cleared.
          Account Name:	admin01
Command : 
Decoded : 

Date    : 13/02/2019 20.01.41
Log     : Security
EventID : 4672
Message : Multiple admin logons for one account
Results : Username: admin01
          User SID Access Count: 3
Command : 
Decoded : 




**Defense Evasion**

Creation of a fake account

In [3]:
Get-ChildItem "Defense_Evasion\" -Filter *.evtx | 
Foreach-Object {
   .\DeepBlue.ps1 $_.FullName
}


Date    : 16/09/2020 12.32.13
Log     : Security
EventID : 4720
Message : New User Created
Results : Username: $
          User SID: S-1-5-21-308926384-506822093-3341789130-107104
          
Command : 
Decoded : 

Date    : 16/09/2020 12.31.19
Log     : Security
EventID : 4720
Message : New User Created
Results : Username: $
          User SID: S-1-5-21-308926384-506822093-3341789130-107103
          
Command : 
Decoded : 




**Persistence**

Network service guest given admin privileges

In [4]:
Get-ChildItem "Persistence\" -Filter *.evtx | 
Foreach-Object {
   .\DeepBlue.ps1 $_.FullName
}


Date    : 22/09/2019 14.23.19
Log     : Security
EventID : 4732
Message : User added to local Administrators group
Results : Username: -
          User SID: S-1-5-20
          
Command : 
Decoded : 

Date    : 22/09/2019 14.22.05
Log     : Security
EventID : 4732
Message : User added to local Administrators group
Results : Username: -
          User SID: S-1-5-21-3461203602-4096304019-2269080069-501
          
Command : 
Decoded : 




### Notes

This process could be automated by running the powershell script on EVTX logs and outputted to CSV or JSON format, which then could be sent forward to a log processing server.

Naturally since DeepBlue is open source, the attacker can see the checks it performs and adjust the methods used in attacks to evade them. However, the tool is a useful part of a larger intrusion detection system and threat hunting process and is useful to be used in tandem with other tools to produce alerts and find indicators of compromise. 